In [42]:
import pandas as pd
import tqdm
import sqlite3 as sq
import matplotlib.pyplot as plt

## Import UIC Data

In [43]:
path = r"C:\Users\Willi\Documents\NYU\2020 Spring\Big Data Science\Project\bds_project\Data"
data_path = path+r"\yelp_dataset_UIC"

res_name = data_path + '\yelpResData.db'
hot_name = data_path + '\yelpHotelData.db'

res_conn = sq.connect(res_name)
hot_conn = sq.connect(hot_name)

In [44]:
res_tables = res_conn.execute("SELECT name FROM sqlite_master WHERE type='table';")
for i,name in enumerate(res_tables):
    print(name[0])
    
print("")

hot_tables = hot_conn.execute("SELECT name FROM sqlite_master WHERE type='table';")
for name in hot_tables:
    print(name[0])

review
restaurant
reviewer

review
sqlite_stat1
sqlite_stat2
reviewer
hotel


In [ ]:
sql_q = lambda x: "SELECT * FROM " + x + ";"
res_conn.text_factory = lambda b: b.decode(errors = 'ignore')
hot_conn.text_factory = lambda b: b.decode(errors = 'ignore')

res_rev_df = pd.read_sql(sql_q('review'),res_conn)
res_rest_df = pd.read_sql(sql_q('restaurant'),res_conn)
res_rever_df = pd.read_sql(sql_q('reviewer'),res_conn)

hot_rev_df = pd.read_sql(sql_q('review'),hot_conn)
hot_sqlite1_df = pd.read_sql(sql_q('sqlite_stat1'),hot_conn)
hot_sqlite2_df = pd.read_sql(sql_q('sqlite_stat2'),hot_conn)
hot_rever_df = pd.read_sql(sql_q('reviewer'),hot_conn)
hot_hot_df = pd.read_sql(sql_q('hotel'),hot_conn)

## Table Columns

In [ ]:
print("restaurant reviews: ",list(res_rev_df.columns.values))
print("restaurant restaurants: ",list(res_rest_df.columns.values))
print("restaurant reviewers: ",list(res_rever_df.columns.values))
print("")
print("hotel reviews: ",list(hot_rev_df.columns.values))
print("hotel sqlite_stat1: ",list(hot_sqlite1_df.columns.values))
print("hotel sqlite_stat2: ",list(hot_sqlite2_df.columns.values))
print("hotel reviewers: ",list(hot_rever_df.columns.values))
print("hotel hotel: ",list(hot_hot_df.columns.values))

In [ ]:
# Find unique values of flagged
res_rev_df.loc[:,'flagged'].unique()

In [ ]:
print(len(res_rev_df['reviewID'].unique()))

In [ ]:
concat = pd.concat([res_rev_df, hot_rev_df])
print('Total number is {}.'.format(concat.shape[0]))
print('Unique number is {}.'.format(len(concat['reviewID'].unique())))
print(concat.shape[0]-len(concat['reviewID'].unique()))

In [ ]:
# First relabel restaurantID and hotelID to businessID
res_rev_df.rename({'restaurantID':'businessID'},axis=1, inplace=True)
hot_rev_df.rename({'hotelID':'businessID'},axis=1, inplace=True)

print(list(res_rev_df.columns.values))
print(list(hot_rev_df.columns.values))

# Remove duplicates
hotel_keep = ~hot_rev_df['reviewID'].isin(res_rev_df['reviewID'])

print(sum(~hotel_keep))

hotel_kept = hot_rev_df.loc[hotel_keep,:].copy()

print(sum(hotel_kept['reviewID'].isin(res_rev_df['reviewID'])))

# Change Y and YR to True and N and NR to False
res_rev_df.loc[res_rev_df['flagged']=='Y',:]=True
res_rev_df.loc[res_rev_df['flagged']=='YR',:]=True
res_rev_df.loc[res_rev_df['flagged']=='N',:]=False
res_rev_df.loc[res_rev_df['flagged']=='NR',:]=False

hotel_kept.loc[hotel_kept['flagged']=='Y',:]=True
hotel_kept.loc[hotel_kept['flagged']=='YR',:]=True
hotel_kept.loc[hotel_kept['flagged']=='N',:]=False
hotel_kept.loc[hotel_kept['flagged']=='NR',:]=False

combined = pd.concat([res_rev_df,hotel_kept]).reset_index(drop=True)

print(res_rev_df['flagged'].unique())
print(combined['flagged'].unique())

res_rev_df.to_csv(path+r'\Cleaned\from_restaurants_reviews_UIC.csv')
combined.to_csv(path+r'\Cleaned\reviews_UIC.csv')

In [ ]:
res_rev_df['has_empty'] = res_rev_df.isna().sum(axis=1)>0

In [ ]:
print(res_rev_df.loc[res_rev_df['has_empty']==False,'has_empty'].shape)

print(res_rev_df.shape)

In [37]:
# Percent flagged
total = res_rev_df.shape[0]

print("N #: ",res_rev_df.loc[res_rev_df.loc[:,'flagged']=='N',:].shape[0])
print("N %: ",res_rev_df.loc[res_rev_df.loc[:,'flagged']=='N',:].shape[0]/total)
print("")
print("Y #: ",res_rev_df.loc[res_rev_df.loc[:,'flagged']=='Y',:].shape[0])
print("Y %: ",res_rev_df.loc[res_rev_df.loc[:,'flagged']=='Y',:].shape[0]/total)
print("")
print("NR #: ",res_rev_df.loc[res_rev_df.loc[:,'flagged']=='NR',:].shape[0])
print("NR %: ",res_rev_df.loc[res_rev_df.loc[:,'flagged']=='NR',:].shape[0]/total)
print("")
print("YR #: ",res_rev_df.loc[res_rev_df.loc[:,'flagged']=='YR',:].shape[0])
print("YR %: ",res_rev_df.loc[res_rev_df.loc[:,'flagged']=='YR',:].shape[0]/total)

N #:  58716
N %:  0.07446817955257708

Y #:  8303
Y %:  0.0105305077802481

NR #:  402774
NR %:  0.5108291871229252

YR #:  318678
YR %:  0.4041721255442496
